<a href="https://colab.research.google.com/github/ArkayNeGG/BigDataUPSO/blob/main/RodriguezGuitleinTavella.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Iniciamos la carga y la concatenación de todos los datasets, limpieza y formateo de los datos

In [ ]:
import os, glob
import pandas as pd

path = "/content/datos_meses"

all_files = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['Mes'] = f.split('_')[4].split('.')[0]
    try:
        df['Precio Unitario'] = df['Precio Unitario'].astype(float)
    except ValueError:
        df['Precio Unitario'] = df['Precio Unitario'].apply(pd.to_numeric, errors='coerce')
    df= df.dropna()
    all_df.append(df)

merged_df = pd.concat(all_df, ignore_index=False)

In [ ]:
merged_df['Cantidad Pedida'] = merged_df['Cantidad Pedida'].astype(int)

### Creamos una función para extraer la hora, fecha y día de la semana de un dataset que tenga una columna registrada con fechas

In [ ]:
def extraer_datos(datos):
  datos['Fecha de Pedido'] = pd.to_datetime(datos['Fecha de Pedido'])
  datos['Hora'] = datos['Fecha de Pedido'].dt.hour
  datos['Dia'] = datos['Fecha de Pedido'].dt.day
  datos['Dia de semana'] = datos['Fecha de Pedido'].dt.strftime('%A')
  mapeo_dias = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
    }
  datos['Dia de semana'] = datos['Dia de semana'].map(mapeo_dias)
  return datos

## Ventas totales en Unidades realizadas por mes

In [ ]:
meses = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
        'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']
totales_ventas = []
nombres_meses = []

for mes in meses:
    dataframe_mes = merged_df.loc[merged_df.Mes == mes]
    total_ventas_mes = dataframe_mes['Cantidad Pedida'].sum()
    totales_ventas.append(total_ventas_mes)
    nombres_meses.append(mes)

In [ ]:
print(totales_ventas)
print(meses)

[10868, 13461, 17006, 20568, 18657, 15255, 16069, 13442, 13119, 22705, 19808, 28121]
['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre']


In [ ]:
import plotly.graph_objects as go

# Creo el objeto figura con el gráfico de barras
fig = go.Figure()

fig.add_trace(go.Bar(
    x=nombres_meses,
    y=totales_ventas,
    marker=dict(color=totales_ventas, colorscale = 'RdYlGn', colorbar=dict(title="Total de ventas", x=1.0, thickness=15)),  # Color y barra de colores
    text=totales_ventas,
    textposition='auto',  # Muestro el valor encima de las barras

))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Total de ventas por mes",
    xaxis=dict(title="Mes"),
    yaxis=dict(title="Total de ventas"),
)

fig.show()

## Ventas concretadas por hora en cada mes

In [ ]:
Horas = list(range(24))

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes = merged_df.loc[merged_df.Mes == mes].copy()
    dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites
    ventas_por_hora = dataframe_mes.groupby('Hora')['Cantidad Pedida'].count()


  # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=Horas,
        y=ventas_por_hora,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora en diferentes meses",
    xaxis=dict(title="Hora", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

## Análisis de cantidad de ventas concretadas en las horas: 12, 15, 18 y 21 de cada mes

In [ ]:
# Creo el objeto figura
fig = go.Figure()
horas_analizadas = [12, 15, 18, 21]

for hora in horas_analizadas:
    total_productos_vendidos = []

    for mes in meses:
        dataframe_mes = merged_df.loc[merged_df.Mes == mes].copy()
        dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites

        ventas_hora = dataframe_mes.loc[dataframe_mes['Hora'] == hora]

        # Calcular la cantidad total de productos vendidos para la hora actual
        total_productos_vendidos.append(ventas_hora['Cantidad Pedida'].count())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=total_productos_vendidos,
        mode='lines',
        name=f'{hora} horas',
        line=dict(width=2, shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades vendidas por hora",
    xaxis=dict(title="Meses", tickvals=Horas),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

## Ventas totales en cada región, en todo el año

In [ ]:
# Separo la direccion de envio en calle ciudad y PO BOX
direccion_df = merged_df['Dirección de Envio'].str.split(', ',expand=True)
direccion_df.columns = ['Calle','Ciudad','PO BOX']

# Separo el PO BOX en Estado y Cod Postal
direccion_df[['Estado', 'Cod Postal']] = direccion_df['PO BOX'].str.split(' ',expand=True)
df_ventas_desglosado = pd.concat([merged_df,direccion_df],axis=True)
df_ventas_desglosado

,ID de Pedido,Producto,Cantidad Pedida,Precio Unitario,Fecha de Pedido,Dirección de Envio,Mes,Calle,Ciudad,PO BOX,Estado,Cod Postal
0,141234,iPhone,1,700.00,01/22/19 21:25,"944 Walnut St, Boston, MA 02215",Enero,944 Walnut St,Boston,MA 02215,MA,02215
1,141235,Lightning Charging Cable,1,14.95,01/28/19 14:15,"185 Maple St, Portland, OR 97035",Enero,185 Maple St,Portland,OR 97035,OR,97035
2,141236,Wired Headphones,2,11.99,01/17/19 13:33,"538 Adams St, San Francisco, CA 94016",Enero,538 Adams St,San Francisco,CA 94016,CA,94016
3,141237,27in FHD Monitor,1,149.99,01/05/19 20:33,"738 10th St, Los Angeles, CA 90001",Enero,738 10th St,Los Angeles,CA 90001,CA,90001
4,141238,Wired Headphones,1,11.99,01/25/19 11:59,"387 10th St, Austin, TX 73301",Enero,387 10th St,Austin,TX 73301,TX,73301
...,...,...,...,...,...,...,...,...,...,...,...,...
25112,319666,Lightning Charging Cable,1,14.95,12/11/19 20:58,"14 Madison St, San Francisco, CA 94016",Diciembre,14 Madison St,San Francisco,CA 94016,CA,94016
25113,319667,AA Batteries (4-pack),2,3.84,12/01/19 12:01,"549 Willow St, Los Angeles, CA 90001",Diciembre,549 Willow St,Los Angeles,CA 90001,CA,90001
25114,319668,Vareebadd Phone,1,400.00,12/09/19 06:43,"273 Wilson St, Seattle, WA 98101",Diciembre,273 Wilson St,Seattle,WA 98101,WA,98101
25115,319669,Wired Headphones,1,11.99,12/03/19 10:39,"778 River St, Dallas, TX 75001",Diciembre,778 River St,Dallas,TX 75001,TX,75001


In [ ]:
# Agrupo por estado y cantidad de productos vendidos
ventas_por_estado = df_ventas_desglosado.groupby('Estado')['Cantidad Pedida'].sum()

# Reseteo el indice
ventas_por_estado = ventas_por_estado.reset_index(inplace=False)
ventas_por_estado

,Estado,Cantidad Pedida
0,CA,83528
1,GA,16602
2,MA,22528
3,ME,2750
4,NY,27932
5,OR,11303
6,TX,27883
7,WA,16553


In [ ]:
import plotly.express as px


# Hago un plot de Estados Unidos
fig = px.choropleth(ventas_por_estado, locations='Estado', locationmode='USA-states',color='Cantidad Pedida',
                           color_continuous_scale="RdYlGn",
                           scope="usa",
                          title= 'Ventas por estado'
                          )

fig.show()

## Ventas realizadas por cada dia de semana en diferentes meses

In [ ]:
semana_dia = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']

# Creo el objeto figura
fig = go.Figure()

for mes in meses:
    dataframe_mes = merged_df.loc[merged_df.Mes == mes].copy()
    dataframe_mes = extraer_datos(dataframe_mes)  # Reemplaza esto con el procesamiento que necesites
    Ventas_por_dia_s = dataframe_mes.groupby('Dia de semana')['Cantidad Pedida'].count()



    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=semana_dia,
        y=Ventas_por_dia_s,
        mode='lines',
        name=mes,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Ventas realizadas por cada dia de semana en diferentes meses",
    xaxis=dict(title="Dia de semana", tickvals=semana_dia),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

fig.show()

## Unidades totales vendidas de cada productos en todo el año

In [ ]:
productos_agrupados = merged_df.loc[:,["Producto", "Cantidad Pedida"]]
productos_total = productos_agrupados.groupby("Producto").sum("Cantidad Pedida").reset_index()
dataframe_ordenado = productos_total.sort_values(by = ['Cantidad Pedida'], ascending = False)
dataframe_ordenado

,Producto,Cantidad Pedida
5,AAA Batteries (4-pack),31017
4,AA Batteries (4-pack),27635
15,USB-C Charging Cable,23975
12,Lightning Charging Cable,23217
17,Wired Headphones,20557
6,Apple Airpods Headphones,15661
7,Bose SoundSport Headphones,13457
2,27in FHD Monitor,7550
18,iPhone,6849
1,27in 4K Gaming Monitor,6244


In [ ]:
# Extraigo valores del dataFrame para realizar los graficos
Productos = dataframe_ordenado['Producto']
Unidades = dataframe_ordenado['Cantidad Pedida']

#Creo el objeto figura
fig = go.Figure()

#Agrego grafico en mi objeto figura
fig.add_trace(go.Bar(
        x = Productos,
        y= Unidades,
        name = 'Unidades Vendidas',
        marker = dict(color= 'Light Blue')
)
)

# Configuro titulos y ejes del grafico
fig.update_layout(
    title = 'Unidades Vendidas',
    xaxis = dict(title='Productos'),
    yaxis = dict(title='Unidades')

)

## Unidades totales vendidas de cada producto, cada mes

In [ ]:
productos_data=[]
for data in dataframe_ordenado['Producto']:
  productos_data.append(data)

In [ ]:
fig = go.Figure()

for producto in productos_data:
    productos_vendidos=[]

    for mes in meses:
        data_mes= merged_df.loc[merged_df['Mes'] == mes].copy()
        data_producto= data_mes.loc[data_mes['Producto'] == producto].copy()
        productos_vendidos.append(data_producto['Cantidad Pedida'].sum())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=productos_vendidos,
        mode='lines',
        name=producto,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades totales vendidas en diferentes meses",
    xaxis=dict(title="Meses", tickvals=meses),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)

## Ventas totales por región, cada mes

In [ ]:
estados=[]
for data in ventas_por_estado['Estado']:
  estados.append(data)

In [ ]:
fig = go.Figure()

for estado in estados:
    productos_vendidos=[]

    for mes in meses:
        data_mes= df_ventas_desglosado.loc[df_ventas_desglosado['Mes'] == mes].copy()
        data_mes_estado= data_mes.loc[data_mes['Estado'] == estado].copy()
        productos_vendidos.append(data_mes_estado['Cantidad Pedida'].sum())

    # Agrego al objeto figura un gráfico de línea con las horas y las ventas por hora
    fig.add_trace(go.Scatter(
        x=meses,
        y=productos_vendidos,
        mode='lines',
        name=estado,
        line=dict(width=2,shape='spline'),
    ))

# Configuro los ejes y los títulos
fig.update_layout(
    title="Unidades totales vendidas en diferentes regiones",
    xaxis=dict(title="Meses", tickvals=meses),
    yaxis=dict(title="Unidades vendidas", side="left"),
    legend=dict(y=0.3, yanchor='bottom'),
)